The goal of this case study is to put into practice the important concepts from module 1.  We will go through the basic process that begins with refining the business opportunity and ensuring that it is articulated using a scientific thought process.

The business opportunity and case study was first mentioned in Unit 2 of module 1 and like the AAVIAL company itself these data were created for learning purposes.  We will be using the AAVAIL example as a basis for this case study.  [Watch the video again](https://vimeo.com/348708995) if you need a refresher.  You will be gathering data from several provided sources, staging it for quality assurance and saving it in a target destination that is most appropriate.

Watch the video to review the important concepts from the units you just covered and to see an overview of the objectives for this case study.

## Case study overall objectives

1. Gather all relevant data from the sources of provided data
2. Implement several checks for quality assurance 
3. Take the initial steps towards automation of the ingestion pipeline

## Getting started

Download this notebook and open it locally using a Jupyter server. Alternatively you may use Watson Studio.  To make using Watson Studio easier we have provided a zip archive file containing the files needed to complete this case study in Watson Studio. See the [Getting started with Watson Studio](m1-u5-5-watson-studio.rst) page.

**You will need the following files to complete this case study**

* [m1-u6-case-study.ipynb](m1-u6-case-study.ipynb)
* [m1-u6-case-study-solution.ipynb](./notebooks/m1-u6-case-study-solution.ipynb)
* [aavail-customers.db](./data/aavail-customers.db)
* [aavail-steams.csv](./data/aavail-streams.csv)

1. Fill in all of the places in this notebook marked with ***YOUR CODE HERE*** or ***YOUR ANSWER HERE***
2. When you have finished the case study there will be a short quiz

You may review the rest of this content as part of the notebook, but once you are ready to get started be ensure that you are working with a *live* version either as part of Watson Studio or locally.

## Data Sources

The data you will be sourcing from is contained in two sources.

1. A database ([SQLite](https://www.sqlite.org/index.html)) of `customer` data
2. A [CSV file](https://en.wikipedia.org/wiki/Comma-separated_values) of `stream` level data

   >You will create a simple data pipeline that
   (1) simplifies the data for future analysis
   (2) performs quality assurance checks.

The process of building *the data ingestion pipeline* entails extracting data, transforming it, and loading it into an appropriate data storage technology.  When constructing a pipeline it is important to keep in mind that they generally process data in batches.  For example, data may be compiled during the day and the batch could be processed during the night.  The data pipeline may also be optimized to execute as a streaming computation (i.e., every event is handled as it occurs).

## PART 1: Gathering the data

The following is an [Entity Relationship Diagram (ERD)](https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model) that details the tables and contents of the database.

<img src="./images/aavail-schema.svg" alt="customer database schema" style="width: 600px;"/>

In [1]:
## all the imports you will need for this case study
import os
import pandas as pd
import numpy as np
import sqlite3

Much of the data exist in a database.  You can connect to is using the `sqlite3` Python package with the function shown below.  Note that is is good practice to wrap your connect functions in a [try-except statement](https://docs.python.org/3/tutorial/errors.html) to cleanly handle exceptions.

In [2]:
def connect_db(file_path):
    try:
        conn = sqlite3.connect(file_path)
        print("...successfully connected to db\n")
    except Error as e:
        print("...unsuccessful connection\n",e)
    
    return(conn)

In [3]:
## make the connection to the database
conn = connect_db('../data_asset/aavail-customers.db')

## print the table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';")]
print(tables)

...successfully connected to db

['CUSTOMER', 'INVOICE', 'INVOICE_ITEM', 'COUNTRY']


### QUESTION 1:

**extract the relevant data from the DB**

Query the database and extract the following data into a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).
 
* Customer ID (integer)
* Last name
* First name
* DOB
* City
* State
* Country (the name NOT the country_id)
* Gender

Remember that that SQL is case-insensitive, but it is traditional to use ALL CAPS for SQL keywords. It is also a convention to end SQL statements with a semi-colon.  

#### Resources

* [W3 schools SQL tutorial](https://www.w3schools.com/sql)
* [W3 schools SQL joins](https://www.w3schools.com/sql/sql_join.asp)

In [4]:
customers = pd.read_sql_query("SELECT customer_id, last_name, first_name, gender, DOB, city, state, country_name \
                              FROM customer LEFT JOIN country ON customer.country_id = country.country_id;", conn)
customers

,customer_id,last_name,first_name,gender,DOB,city,state,country_name
0,1,Todd,Kasen,m,07/30/98,Rock Hill,South Carolina,united_states
1,2,Garza,Ensley,f,04/12/89,singapore,None,singapore
2,3,Carey,Lillian,f,09/12/97,Auburn,Alabama,united_states
3,4,Christensen,Beau,m,01/28/99,Hempstead,New York,united_states
4,5,Gibson,Ernesto,m,03/23/98,singapore,None,singapore
...,...,...,...,...,...,...,...,...
1002,21,Hickman,Claudia,f,06/08/81,singapore,None,singapore
1003,31,Rubio,Dalary,f,07/12/00,Indio,California,united_states
1004,301,Fisher,Jamie,m,12/31/82,singapore,None,singapore
1005,401,Hill,Brodie,m,11/30/98,singapore,None,singapore


### QUESTION 2:

**Extract the relevant data from the CSV file**

For each ```customer_id``` determine if a customer has stopped their subscription or not and save it in a dictionary or another data container.

In [5]:
df_streams = pd.read_csv('../data_asset/aavail-streams.csv')
df_streams

,customer_id,stream_id,date,invoice_item_id,subscription_stopped
0,1,1420.0,2018-10-21,2.0,0
1,1,1343.0,2018-10-23,2.0,0
2,1,1756.0,2018-11-05,2.0,0
3,1,1250.0,2018-11-06,2.0,0
4,1,1324.0,2018-11-12,2.0,0
...,...,...,...,...,...
18854,1000,1729.0,2019-07-01,3.0,0
18855,1000,1527.0,2019-07-10,3.0,0
18856,1000,1254.0,2019-07-10,3.0,0
18857,1000,1356.0,2019-07-12,3.0,0


In [6]:
def is_sub(row):
    is_sub = df_streams.loc[df_streams['customer_id'] == row['customer_id']]['subscription_stopped'].sum()
    return is_sub

In [7]:
customers['is_sub'] = customers.apply(is_sub, axis = 1)
customers

,customer_id,last_name,first_name,gender,DOB,city,state,country_name,is_sub
0,1,Todd,Kasen,m,07/30/98,Rock Hill,South Carolina,united_states,0
1,2,Garza,Ensley,f,04/12/89,singapore,None,singapore,1
2,3,Carey,Lillian,f,09/12/97,Auburn,Alabama,united_states,1
3,4,Christensen,Beau,m,01/28/99,Hempstead,New York,united_states,0
4,5,Gibson,Ernesto,m,03/23/98,singapore,None,singapore,0
...,...,...,...,...,...,...,...,...,...
1002,21,Hickman,Claudia,f,06/08/81,singapore,None,singapore,1
1003,31,Rubio,Dalary,f,07/12/00,Indio,California,united_states,0
1004,301,Fisher,Jamie,m,12/31/82,singapore,None,singapore,1
1005,401,Hill,Brodie,m,11/30/98,singapore,None,singapore,0


## PART 2: Checks for quality assurance

Sometimes it is known in advance which types of data integrity issues to expect, but other times it is during the Exploratory Data Analysis (EDA) process that these issues are identified.  After extracting data it is important to include checks for quality assurance even on the first pass through the AI workflow.  Here you will combine the data into a single structure and provide a couple checks for quality assurance.

### QUESTION 3: 

**Implement checks for quality assurance**

1. Remove any repeat customers based on ```customer_id```
2. Remove stream data that do not have an associated ```stream_id```
3. Check for missing values

In [8]:
customers = customers.drop_duplicates()
customers

,customer_id,last_name,first_name,gender,DOB,city,state,country_name,is_sub
0,1,Todd,Kasen,m,07/30/98,Rock Hill,South Carolina,united_states,0
1,2,Garza,Ensley,f,04/12/89,singapore,None,singapore,1
2,3,Carey,Lillian,f,09/12/97,Auburn,Alabama,united_states,1
3,4,Christensen,Beau,m,01/28/99,Hempstead,New York,united_states,0
4,5,Gibson,Ernesto,m,03/23/98,singapore,None,singapore,0
...,...,...,...,...,...,...,...,...,...
995,996,Enriquez,Peyton,m,12/08/64,singapore,None,singapore,1
996,997,Manning,Amina,f,08/06/96,Dublin,California,united_states,0
997,998,Ventura,Brooks,m,01/26/95,St. Clair Shores,Michigan,united_states,0
998,999,Mathis,Nayeli,f,01/28/80,Pleasant Grove,Utah,united_states,0


In [9]:
print(df_streams.isnull().sum())

customer_id                0
stream_id               1164
date                       0
invoice_item_id            0
subscription_stopped       0
dtype: int64


In [10]:
df_streams = df_streams.dropna(how = 'any', axis = 0)
df_streams

,customer_id,stream_id,date,invoice_item_id,subscription_stopped
0,1,1420.0,2018-10-21,2.0,0
1,1,1343.0,2018-10-23,2.0,0
2,1,1756.0,2018-11-05,2.0,0
3,1,1250.0,2018-11-06,2.0,0
4,1,1324.0,2018-11-12,2.0,0
...,...,...,...,...,...
18854,1000,1729.0,2019-07-01,3.0,0
18855,1000,1527.0,2019-07-10,3.0,0
18856,1000,1254.0,2019-07-10,3.0,0
18857,1000,1356.0,2019-07-12,3.0,0


In [11]:
customers.isnull().sum()

customer_id       0
last_name         0
first_name        0
gender            0
DOB               0
city              0
state           300
country_name      0
is_sub            0
dtype: int64

### QUESTION 4: 

**combine the data into a single data structure**

For this example, the two most convenient structures for this task are Pandas dataframes and NumPy arrays.  At a minimum ensure that your structure accommodates the following.

1. A column for `customer_id`
2. A column for `country`
3. A column for ```age``` that is created from ```DOB```
4. A column ```customer_name``` that is created from ```first_name``` and ```last_name```
5. A column to indicate churn called ```is_subscriber```
7. A column that indicates ```subscriber_type``` that comes from ```invoice_item```
6. A column to indicate the total ```num_streams```

#### Resources

* [Python's datetime library](https://docs.python.org/3/library/datetime.html)
* [NumPy's datetime data type](https://docs.scipy.org/doc/numpy/reference/arrays.datetime.html)


In [12]:
customers

,customer_id,last_name,first_name,gender,DOB,city,state,country_name,is_sub
0,1,Todd,Kasen,m,07/30/98,Rock Hill,South Carolina,united_states,0
1,2,Garza,Ensley,f,04/12/89,singapore,None,singapore,1
2,3,Carey,Lillian,f,09/12/97,Auburn,Alabama,united_states,1
3,4,Christensen,Beau,m,01/28/99,Hempstead,New York,united_states,0
4,5,Gibson,Ernesto,m,03/23/98,singapore,None,singapore,0
...,...,...,...,...,...,...,...,...,...
995,996,Enriquez,Peyton,m,12/08/64,singapore,None,singapore,1
996,997,Manning,Amina,f,08/06/96,Dublin,California,united_states,0
997,998,Ventura,Brooks,m,01/26/95,St. Clair Shores,Michigan,united_states,0
998,999,Mathis,Nayeli,f,01/28/80,Pleasant Grove,Utah,united_states,0


In [13]:
customers['name'] = customers['first_name'] + ' ' + customers['last_name']
customers = customers.drop(['first_name', 'last_name'], axis = 1)
customers

C:\Source-Files\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,customer_id,gender,DOB,city,state,country_name,is_sub,name
0,1,m,07/30/98,Rock Hill,South Carolina,united_states,0,Kasen Todd
1,2,f,04/12/89,singapore,None,singapore,1,Ensley Garza
2,3,f,09/12/97,Auburn,Alabama,united_states,1,Lillian Carey
3,4,m,01/28/99,Hempstead,New York,united_states,0,Beau Christensen
4,5,m,03/23/98,singapore,None,singapore,0,Ernesto Gibson
...,...,...,...,...,...,...,...,...
995,996,m,12/08/64,singapore,None,singapore,1,Peyton Enriquez
996,997,f,08/06/96,Dublin,California,united_states,0,Amina Manning
997,998,m,01/26/95,St. Clair Shores,Michigan,united_states,0,Brooks Ventura
998,999,f,01/28/80,Pleasant Grove,Utah,united_states,0,Nayeli Mathis


In [14]:
import datetime
def age(row):
    dob = datetime.datetime.strptime(row['DOB'], '%m/%d/%y').date()
    today = datetime.datetime.today
    diff = datetime.date.today() - dob
    return int(diff.days / 365)

customers['age'] = customers.apply(age, axis = 1)
customers = customers.drop(['DOB'], axis = 1)
customers

,customer_id,gender,city,state,country_name,is_sub,name,age
0,1,m,Rock Hill,South Carolina,united_states,0,Kasen Todd,21
1,2,f,singapore,None,singapore,1,Ensley Garza,31
2,3,f,Auburn,Alabama,united_states,1,Lillian Carey,22
3,4,m,Hempstead,New York,united_states,0,Beau Christensen,21
4,5,m,singapore,None,singapore,0,Ernesto Gibson,22
...,...,...,...,...,...,...,...,...
995,996,m,singapore,None,singapore,1,Peyton Enriquez,-44
996,997,f,Dublin,California,united_states,0,Amina Manning,23
997,998,m,St. Clair Shores,Michigan,united_states,0,Brooks Ventura,25
998,999,f,Pleasant Grove,Utah,united_states,0,Nayeli Mathis,40


In [15]:
def count_streams(row):
    stream_count = df_streams.loc[df_streams['customer_id'] == row['customer_id']]
    return len(stream_count)

customers['num_streams'] = customers.apply(count_streams, axis = 1)
customers

,customer_id,gender,city,state,country_name,is_sub,name,age,num_streams
0,1,m,Rock Hill,South Carolina,united_states,0,Kasen Todd,21,23
1,2,f,singapore,None,singapore,1,Ensley Garza,31,12
2,3,f,Auburn,Alabama,united_states,1,Lillian Carey,22,22
3,4,m,Hempstead,New York,united_states,0,Beau Christensen,21,19
4,5,m,singapore,None,singapore,0,Ernesto Gibson,22,23
...,...,...,...,...,...,...,...,...,...
995,996,m,singapore,None,singapore,1,Peyton Enriquez,-44,14
996,997,f,Dublin,California,united_states,0,Amina Manning,23,24
997,998,m,St. Clair Shores,Michigan,united_states,0,Brooks Ventura,25,17
998,999,f,Pleasant Grove,Utah,united_states,0,Nayeli Mathis,40,16


In [16]:
count = 1
def sub_type(row):
    global count
    sub_type = df_streams.loc[df_streams['customer_id'] == row['customer_id']]
    return int(sub_type['invoice_item_id'].iloc[-1])
    
customers['sub_type'] = customers.apply(sub_type, axis = 1)
customers

,customer_id,gender,city,state,country_name,is_sub,name,age,num_streams,sub_type
0,1,m,Rock Hill,South Carolina,united_states,0,Kasen Todd,21,23,2
1,2,f,singapore,None,singapore,1,Ensley Garza,31,12,3
2,3,f,Auburn,Alabama,united_states,1,Lillian Carey,22,22,2
3,4,m,Hempstead,New York,united_states,0,Beau Christensen,21,19,1
4,5,m,singapore,None,singapore,0,Ernesto Gibson,22,23,2
...,...,...,...,...,...,...,...,...,...,...
995,996,m,singapore,None,singapore,1,Peyton Enriquez,-44,14,3
996,997,f,Dublin,California,united_states,0,Amina Manning,23,24,1
997,998,m,St. Clair Shores,Michigan,united_states,0,Brooks Ventura,25,17,3
998,999,f,Pleasant Grove,Utah,united_states,0,Nayeli Mathis,40,16,3


In [17]:
customers.sub_type.value_counts()

1    367
2    331
3    302
Name: sub_type, dtype: int64

In [18]:
customer_data = customers.drop(['gender', 'city', 'state'], axis = 1)
customer_data

,customer_id,country_name,is_sub,name,age,num_streams,sub_type
0,1,united_states,0,Kasen Todd,21,23,2
1,2,singapore,1,Ensley Garza,31,12,3
2,3,united_states,1,Lillian Carey,22,22,2
3,4,united_states,0,Beau Christensen,21,19,1
4,5,singapore,0,Ernesto Gibson,22,23,2
...,...,...,...,...,...,...,...
995,996,singapore,1,Peyton Enriquez,-44,14,3
996,997,united_states,0,Amina Manning,23,24,1
997,998,united_states,0,Brooks Ventura,25,17,3
998,999,united_states,0,Nayeli Mathis,40,16,3


## PART 3: Automating the process

To ensure that you code can be used to automate this process.  First you will save you dataframe or numpy array as a CSV file.  

### QUESTION 5:

**Take the initial steps towards automation**

1. Save your cleaned, combined data as a CSV file.
2. From the code above create a function or class that performs all of the steps given a database file and a streams CSV file.
3. Run the function in batches and write a check to ensure you got the same result that you did in the code above.

There will be some logic involved to ensure that you do not write the same data twice to the target CSV file.

Shown below is some code that will split your streams file into two batches. 

In [19]:
# body = client_bfb337e9d3b24554ad9a816fe610a744.get_object(Bucket='aienterpriseworkflowmodule1casest-donotdelete-pr-qwt6qhnjsctfni',Key='aavail-streams.csv')['Body']
# # add missing __iter__ method, so pandas accepts body as file-like object
# if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# ## code to split the streams csv into batches

# df_all = pd.read_csv(body)
# half = int(round(df_all.shape[0] * 0.5))
# df_part1 = df_all[:half]
# df_part2 = df_all[half:]
# df_part1.to_csv = (os.path.join(".","aavail-streams-1.csv"))
# df_part2.to_csv = (os.path.join(".","aavail-streams-2.csv"))

You will need to save your function as a file.  The following cell demonstrates how to do this from within a notebook. 

In [20]:
%%writefile aavail-data-ingestor.py
#!/usr/bin/env python

import os
import pandas as pd
import numpy as np
import sqlite3

data_dir = os.path.join(".","data")

pass

Overwriting aavail-data-ingestor.py


You will also need to be able to pass the file names to your function without hardcoding them into the script itself.  This is an important step towards automation.  Here are the two libraries commonly used to accomplish this in Python.

* [getopt](https://docs.python.org/3/library/getopt.html)
* [argparse](https://docs.python.org/3/library/argparse.html)

You may run the script you just created from the commandline directly or from within this notebook using:

In [21]:
!python aavail-data-ingestor.py aavail-customers.db aavail-streams-1.csv

Run the script once for each batch that you created and then load both the original and batch versions back into the notebook to check that they are the same. 

In [23]:
class Aavail_Data_Ingestor():
    def __init__(self, connection_path = None, df_stream_path = None, pd_query = None, automate = False):
        import sqlite3
        import datetime
        import pandas as pd
        self.connection = None
        self.customers_original = None
        self.customers = None
        self.df_streams_original = None
        self.df_streams = None
        self.customers_processed = None
        self.df_streams_processed = None
        
        if pd_query != None and connection_path != None:
            self.connect_db(connection_path)
            self.set_customers(pd_query)
        elif pd_query == None and connection_path != None:
            self.connect_db(connection_path)
        
        if df_stream_path != None:
            self.set_df_streams(df_stream_path)
        
        if automate:
#             assert self.df_streams != None or self.customers != None,\
#                 "in order to automate the process you need a proper, \
#                 connection_path, df_stream_path, and pd_query"
            self.begin_clean_pipeline()
    # Setters
    def connect_db(self, file_path):
        try:
            self.connection = sqlite3.connect(file_path)
            print("...successfully connected to db\n")
        except Error as e:
            print("...unsuccessful connection\n",e)
            
    def set_customers(self, pd_query):
        self.customers_original = pd.read_sql_query(pd_query, self.connection)
        self.customers = self.customers_original.copy()
        
    def set_df_streams(self, file_path):
        self.df_streams_original = pd.read_csv(file_path)
        self.df_streams = pd.read_csv(file_path)
    # Resetter
    def wipe_data(self, original = False, working_data = False, processed = False):
        if original:
            self.customers_original
            self.df_streams_original
        if working_data:
            self.customers
            self.df_streams
        if processed:
            self.customers_processed
            self.df_streams_processed
        
    # The pipeline - - - - - - - - - -
    def set_is_sub(self, cu, st):
        def sub_apply(row):
            return st.loc[st['customer_id'] == row['customer_id']]['subscription_stopped'].sum()
        cu['is_sub'] = cu.apply(sub_apply, axis = 1)
        return cu

    def drop_duplicate_rows(self, cu):
        return cu.drop_duplicates()

    def drop_na(self, st):
        return st.dropna(how = 'any', axis = 0)

    def set_name(self, cu):
        cu['name'] = cu['first_name'] + ' ' + cu['last_name']
        return cu.drop(['first_name', 'last_name'], axis = 1)

    def set_age(self, cu):
        def age_apply(row):
            dob = datetime.datetime.strptime(row['DOB'], '%m/%d/%y').date()
            today = datetime.datetime.today
            diff = datetime.date.today() - dob
            return int(diff.days / 365)
        cu['age'] = cu.apply(age_apply, axis = 1)
        return cu.drop(['DOB'], axis = 1)

    def set_num_streams(self, cu, st):
        def num_streams_apply(row):
            stream_count = st.loc[st['customer_id'] == row['customer_id']]
            return len(stream_count)
        cu['num_streams'] = cu.apply(num_streams_apply, axis = 1)
        return cu

    def set_sub_type(self, cu, st):
        def sub_type_apply(row):
            sub_type = st.loc[st['customer_id'] == row['customer_id']]
            return int(sub_type['invoice_item_id'].iloc[-1])
        cu['sub_type'] = cu.apply(sub_type_apply, axis = 1)
        return cu
    # End the pipeline - - - - - - - -
    
    def begin_clean_pipeline(self):
#         assert self.df_streams == None or self.customers == None, \
#             "unable to begin pipeline, customers and/or df_streams are uninitialized"
        cu = self.customers.copy()
        st = self.df_streams.copy()
        # The pipeline
        cu = self.set_is_sub(cu, st)
        print('set_is_sub, complete...')
        cu = self.drop_duplicate_rows(cu)
        print('drop_duplicate_rows, complete...')
        st = self.drop_na(st)
        print('drop_na, complete...')
        cu = self.set_name(cu)
        print('set_name, complete...')
        cu = self.set_age(cu)
        print('set_age, complete...')
        cu = self.set_num_streams(cu, st)
        print('set_num_streams, complete...')
        cu = self.set_sub_type(cu, st)
        print('set_sub_type, complete...')
        # End the pipeline
        self.customers_processed = cu
        self.df_streams_processed = st
        self.wipe_data(original = True, working_data = True)
        print('pipline, complete. returning')
        return cu, st

In [24]:
pandas_sql_query = "\
            SELECT customer_id, last_name, \
            first_name, gender, DOB, city, \
            state, country_name FROM customer \
            LEFT JOIN country ON customer.country_id = \
            country.country_id;"
df_streams_path = '../data_asset/aavail-streams.csv'
connection_path = '../data_asset/aavail-customers.db'

In [25]:
# Testing...
aavail_di = Aavail_Data_Ingestor(connection_path, df_streams_path, pandas_sql_query, False)

...successfully connected to db



In [26]:
aavail_di.customers

,customer_id,last_name,first_name,gender,DOB,city,state,country_name
0,1,Todd,Kasen,m,07/30/98,Rock Hill,South Carolina,united_states
1,2,Garza,Ensley,f,04/12/89,singapore,None,singapore
2,3,Carey,Lillian,f,09/12/97,Auburn,Alabama,united_states
3,4,Christensen,Beau,m,01/28/99,Hempstead,New York,united_states
4,5,Gibson,Ernesto,m,03/23/98,singapore,None,singapore
...,...,...,...,...,...,...,...,...
1002,21,Hickman,Claudia,f,06/08/81,singapore,None,singapore
1003,31,Rubio,Dalary,f,07/12/00,Indio,California,united_states
1004,301,Fisher,Jamie,m,12/31/82,singapore,None,singapore
1005,401,Hill,Brodie,m,11/30/98,singapore,None,singapore


In [27]:
aavail_di.df_streams

,customer_id,stream_id,date,invoice_item_id,subscription_stopped
0,1,1420.0,2018-10-21,2.0,0
1,1,1343.0,2018-10-23,2.0,0
2,1,1756.0,2018-11-05,2.0,0
3,1,1250.0,2018-11-06,2.0,0
4,1,1324.0,2018-11-12,2.0,0
...,...,...,...,...,...
18854,1000,1729.0,2019-07-01,3.0,0
18855,1000,1527.0,2019-07-10,3.0,0
18856,1000,1254.0,2019-07-10,3.0,0
18857,1000,1356.0,2019-07-12,3.0,0


In [28]:
c, s = aavail_di.begin_clean_pipeline()

set_is_sub, complete...
drop_duplicate_rows, complete...
drop_na, complete...
set_name, complete...
set_age, complete...
set_num_streams, complete...
set_sub_type, complete...
pipline, complete. returning


In [29]:
c

,customer_id,gender,city,state,country_name,is_sub,name,age,num_streams,sub_type
0,1,m,Rock Hill,South Carolina,united_states,0,Kasen Todd,21,23,2
1,2,f,singapore,None,singapore,1,Ensley Garza,31,12,3
2,3,f,Auburn,Alabama,united_states,1,Lillian Carey,22,22,2
3,4,m,Hempstead,New York,united_states,0,Beau Christensen,21,19,1
4,5,m,singapore,None,singapore,0,Ernesto Gibson,22,23,2
...,...,...,...,...,...,...,...,...,...,...
995,996,m,singapore,None,singapore,1,Peyton Enriquez,-44,14,3
996,997,f,Dublin,California,united_states,0,Amina Manning,23,24,1
997,998,m,St. Clair Shores,Michigan,united_states,0,Brooks Ventura,25,17,3
998,999,f,Pleasant Grove,Utah,united_states,0,Nayeli Mathis,40,16,3


In [30]:
s

,customer_id,stream_id,date,invoice_item_id,subscription_stopped
0,1,1420.0,2018-10-21,2.0,0
1,1,1343.0,2018-10-23,2.0,0
2,1,1756.0,2018-11-05,2.0,0
3,1,1250.0,2018-11-06,2.0,0
4,1,1324.0,2018-11-12,2.0,0
...,...,...,...,...,...
18854,1000,1729.0,2019-07-01,3.0,0
18855,1000,1527.0,2019-07-10,3.0,0
18856,1000,1254.0,2019-07-10,3.0,0
18857,1000,1356.0,2019-07-12,3.0,0


### QUESTION 6:

**How can you improve the process?**

In paragraph form or using bullets write down some of the ways that you could improve this pipeline.

YOUR ANSWER HERE




* By making it more customizable, i.e. having different pipelines for different things.
* Make it possible to use just SQL instead of python dataframes
* Implement optparser
* Implement optparser/argparser with a verbose and quiet option
* Make it so that the processed data can be sent to a database